In [1]:
# dependencies

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import datetime
import plotly.express as px

from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)

In [2]:
# database connection

db_password = "UnitCircle42!"
db_user = "postgres"
db_name = "dot"
endpoint = "awakedb.cre3f7yk1unp.us-west-1.rds.amazonaws.com"

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

In [137]:
# IF EXCEL
file_path = r"C:\Users\mikej\Master Data\cpg-sales\data\unl_2024_02(incomplete).xlsx"

new_unl_raw = pd.read_excel(file_path,header=2)
new_unl_raw.drop(new_unl_raw.tail(1).index,inplace=True)
new_unl_raw.drop(columns='Unnamed: 9',inplace=True)
print(f"{len(new_unl_raw)} new rows")
new_unl_raw.columns

91 new rows


Index(['Order No.', 'Order Date', 'Required Date', 'Completed Date',
       'Warehouse', 'Customer', 'Customer Type', 'Product', 'Product Group',
       'Status', 'Quantity', 'Sub Total'],
      dtype='object')

In [138]:
# FIX HEADERS, DATATYPES, AND SEND NEW ROWS TO DATABASE

new_unl_raw.columns = ['order_num','order_date','req_date','completed_date','warehouse','customer_name','customer_type','product','product_group','status','quantity','sub_total']
new_unl_raw['quantity'] = new_unl_raw['quantity'].astype('float')
new_unl_raw['sub_total'] = new_unl_raw['sub_total'].astype('float')
new_unl_raw.to_sql('unleashed_raw', engine, if_exists='append', index=False)

91

In [3]:
unleashed_raw = pd.read_sql('SELECT * FROM unleashed_raw;', con = engine)

unleashed_raw.drop(columns=['order_num','order_date','req_date','warehouse','customer_type','product_group','status'],inplace=True)

# ## add $ USD columns converted from CAD
unleashed_raw['usd'] = round(unleashed_raw['sub_total']*.75,2)
# ## add origin of sale (dot or unleashed)
unleashed_raw['sale_origin'] = 'unl'
# ## add market segments to align with dot segments
unleashed_raw['market_segment'] = 'direct'

# # add parent distributor column
# parent_child_table = pd.read_excel(r'C:\Users\mikej\Master Data\cpg-sales\data\parent_child.xlsx')

# temp_unl = unleashed_raw.set_index('customer_name')
# parent_child_table = parent_child_table.set_index('customer_name')
# unleashed_raw = pd.merge(temp_unl,parent_child_table,how='left',left_index=True,right_index=True).reset_index()

# # # unleashed_raw.dropna(subset = ['parent_customer'], inplace=True)

# unleashed_raw.parent_customer.fillna(unleashed_raw['customer_name'], inplace=True)

print(len(unleashed_raw))
unleashed_raw.info()

197018
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197018 entries, 0 to 197017
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   completed_date  197018 non-null  object 
 1   customer_name   197018 non-null  object 
 2   product         197018 non-null  object 
 3   quantity        197018 non-null  float64
 4   sub_total       197018 non-null  float64
 5   usd             197018 non-null  float64
 6   sale_origin     197018 non-null  object 
 7   market_segment  197018 non-null  object 
dtypes: float64(3), object(5)
memory usage: 12.0+ MB


In [4]:
unleashed_raw.customer_name.nunique()

956

In [55]:
parent_child_table = pd.read_excel(r'C:\Users\mikej\Master Data\cpg-sales\data\parent_child.xlsx')

temp_unl = unleashed_raw.set_index('customer_name')
parent_child_table = parent_child_table.set_index('customer_name')
# tempy = pd.merge(temp_unl,parent_child_table,how='left',left_index=True,right_index=True).reset_index()
parent_child_table
tempy = pd.merge(temp_unl,parent_child_table,how='inner',left_index=True,right_index=True).reset_index()
tempy.drop_duplicates()

,customer_name,completed_date,product,quantity,sub_total,usd,sale_origin,market_segment,parent_customer
0,ACME,2018-01-12,Shopify Shipping cost,1.0,6.39,4.79,unl,direct,ACME
1,ACME,2018-01-12,50ct Change Maker - AWAKE Chocolate Bites USA ...,1.0,38.32,28.74,unl,direct,ACME
2,"AMG Global Distribution, Inc.",2019-02-28,6 x 12 Pack - AWAKE Caffeinated Chocolate Bars...,24.0,1719.29,1289.47,unl,direct,AMG Global Distribution
3,"AMG Global Distribution, Inc.",2019-02-28,6 x 12 Pack - AWAKE Caffeinated Chocolate Bars...,144.0,10315.73,7736.80,unl,direct,AMG Global Distribution
4,"AMG Global Distribution, Inc.",2019-02-28,6 x 12 Pack - AWAKE Caffeinated Chocolate Bars...,144.0,10315.73,7736.80,unl,direct,AMG Global Distribution
...,...,...,...,...,...,...,...,...,...
388305,na,2018-01-12,Caramel Chocolate Bars 12 Pack,1.0,20.43,15.32,unl,direct,Consumer
388306,na,2018-01-12,Dark Chocolate Bars 12 Pack,1.0,20.43,15.32,unl,direct,Consumer
388307,na,2018-01-12,Dark Chocolate Peanut Butter Granola Bars 16 Pack,1.0,25.54,19.16,unl,direct,Consumer
388308,online seller,2018-02-07,Milk Chocolate Bars 12 Pack,3.0,61.31,45.98,unl,direct,online seller


In [33]:
print(unleashed_raw.completed_date.min())
print(unleashed_raw.completed_date.max())
print(f"Rows: {len(unleashed_raw)}")
unleashed_raw.head(3)

2018-01-05
2024-01-09
Rows: 197018


,completed_date,customer_name,product,quantity,sub_total,usd,sale_origin,market_segment
0,2021-03-10,GoJava,Shipping Cost (Huzzah! Free shipping achieved!),1.0,0.0,0.0,unl,direct
1,2021-03-10,GoJava,6 x 12 Pack - AWAKE Caffeinated Chocolate Bars...,4.0,248.0,186.0,unl,direct
2,2021-03-07,GoJava,Shipping Cost (Huzzah! Free shipping achieved!),1.0,0.0,0.0,unl,direct


In [145]:
# drop unnecessary columns

new_unl_raw.drop(columns=['order_num','order_date','req_date','warehouse','customer_type','product_group','status'],inplace=True)

## add $ USD columns converted from CAD
new_unl_raw['usd'] = round(new_unl_raw['sub_total']*.75,2)
## add origin of sale (dot or unleashed)
new_unl_raw['sale_origin'] = 'unl'
## add market segments to align with dot segments
new_unl_raw['market_segment'] = 'direct'



## assign parent customer to each customer_name
new_unl_raw['parent_customer'] = 'customer_name'




# add year/month columns
new_unl_raw['completed_date'] = pd.to_datetime(new_unl_raw['completed_date'])
year_col = new_unl_raw.set_index(['completed_date']).index.year
month_col = new_unl_raw.set_index(['completed_date']).index.month_name()

# month & year columns
new_unl_raw.insert(0,"month", month_col)
new_unl_raw.insert(1,"year", year_col)

new_unl_raw = new_unl_raw.convert_dtypes()
new_unl_raw.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   month            91 non-null     string        
 1   year             91 non-null     Int32         
 2   completed_date   91 non-null     datetime64[ns]
 3   customer_name    91 non-null     string        
 4   product          91 non-null     string        
 5   quantity         91 non-null     Int64         
 6   sub_total        91 non-null     Float64       
 7   usd              91 non-null     Float64       
 8   sale_origin      91 non-null     string        
 9   market_segment   91 non-null     string        
 10  parent_customer  91 non-null     string        
dtypes: Float64(2), Int32(1), Int64(1), datetime64[ns](1), string(6)
memory usage: 7.9 KB


In [148]:
##  -- SEND TO UNLEASHED CLEAN DATABASE

new_unl_raw.to_sql('unleashed_clean', engine, if_exists='append', index=False)

91

In [3]:
## read in unleashed_clean

unleashed_clean = pd.read_sql('SELECT * FROM unleashed_clean;', con = engine)

print(unleashed_clean.completed_date.min())
print(unleashed_clean.completed_date.max())
print(f"Rows: {len(unleashed_clean)}")
unleashed_clean.head(3)

2018-01-05 00:00:00
2024-01-09 13:55:41.081000
Rows: 197018


,month,year,completed_date,customer_name,product,quantity,sub_total,usd,sale_origin,market_segment,parent_customer
0,March,2021,2021-03-10,GoJava,Shipping Cost (Huzzah! Free shipping achieved!),1.0,0.0,0.0,unl,direct,customer_name
1,March,2021,2021-03-10,GoJava,6 x 12 Pack - AWAKE Caffeinated Chocolate Bars...,4.0,248.0,186.0,unl,direct,customer_name
2,March,2021,2021-03-07,GoJava,Shipping Cost (Huzzah! Free shipping achieved!),1.0,0.0,0.0,unl,direct,customer_name


In [16]:
# df = unleashed_clean[['customer_name','parent_customer']]
# df.to_csv('unl_parent_list.csv')
# df = pd.read_sql('SELECT * FROM unleashed_raw;', con = engine)
df['parent_customer'] = 'parent_customer'
df = df[['customer_name','parent_customer']]
df.to_csv('unl_customer_list.csv')